# Chatbot Trainer file

Thank you to https://dzone.com/articles/python-chatbot-project-build-your-first-python-pro for the inspiration here. 

In [6]:
# Important imports
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer 
import numpy as np
import random
import nltk
import json
import pickle

# Download 'wordnet'
nltk.download('wordnet')

# Initializing Lemmatizer
lemmatizer = WordNetLemmatizer()

# Reading in intents.json file
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rob\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [7]:
words = []
classes = []
documents = []
ignore = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)

        # Add documents in the corpus
        documents.append((word, intent['tag']))

        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [8]:
# Lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Documents = combination between patterns and intents
print (len(documents), "documents")

# Classes = intents
print (len(classes), "classes", classes)

# Words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [9]:
# Create the training data
training = []

# Create empty array for the output
output_empty = [0] * len(classes)

# Training set, bag of words for every sentence
for doc in documents:

    # Initializing bag of words
    bag = []

    # List of tokenized words for the pattern
    word_patterns = doc[0]

    # Lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    # Create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)   

    # Output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)

# Create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data is created")

Training data is created


In [10]:
# Deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model is created")

 1.6376 - accuracy: 0.5532
Epoch 7/200
10/10 [==============================] - 0s 700us/step - loss: 1.4401 - accuracy: 0.5745
Epoch 8/200
10/10 [==============================] - 0s 800us/step - loss: 1.4499 - accuracy: 0.5319
Epoch 9/200
10/10 [==============================] - 0s 800us/step - loss: 1.2950 - accuracy: 0.5957
Epoch 10/200
10/10 [==============================] - 0s 800us/step - loss: 1.3130 - accuracy: 0.6383
Epoch 11/200
10/10 [==============================] - 0s 800us/step - loss: 1.0381 - accuracy: 0.7447
Epoch 12/200
10/10 [==============================] - 0s 700us/step - loss: 1.0141 - accuracy: 0.6809
Epoch 13/200
10/10 [==============================] - 0s 900us/step - loss: 0.8927 - accuracy: 0.7660
Epoch 14/200
10/10 [==============================] - 0s 800us/step - loss: 1.0490 - accuracy: 0.6170
Epoch 15/200
10/10 [==============================] - 0s 700us/step - loss: 0.8686 - accuracy: 0.8511
Epoch 16/200
10/10 [==============================] - 0s 9